In [1]:
#Auxiliares
import os
import pickle
from time import time
from datetime import datetime
from json import dumps

#Dados
import pandas as pd


#Machine learning
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MaxAbsScaler
#Metrics
from sklearn.metrics import classification_report

seed = 42

In [2]:
# Mount Driver
#drive.mount('/content/gdrive')

In [3]:
#Variables
base_path = 'C:/Users/arthu/Desktop/22032020 - Experimentos/05. Organizado/03. Datasets/Atendimento-Unbalanced-Binary'
save_path = 'output'

label='Atendimento'

x_train_file = 'X_train.csv'
y_train_file = 'y_train.csv'
x_test_file = 'X_test.csv'
y_test_file = 'y_test.csv'

#Load data
X_train = pd.read_csv(os.path.join(base_path, x_train_file), sep=';', encoding='utf-8')
y_train = pd.read_csv(os.path.join(base_path, y_train_file), sep=';', encoding='utf-8')
X_test = pd.read_csv(os.path.join(base_path, x_test_file), sep=';', encoding='utf-8')
y_test = pd.read_csv(os.path.join(base_path, y_test_file), sep=';', encoding='utf-8')

#Checking on data
print(X_train.columns)
print(X_train.shape)
print(y_train[label].value_counts())
print(y_test[label].value_counts())

Index(['pid', 'req-text', 'resp-text', '1funct-request', '2pronoun-request',
       '3ppron-request', '4i-request', '5we-request', '6you-request',
       '7shehe-request',
       ...
       '58home-response', '59money-response', '60relig-response',
       '61death-response', '62assent-response', '63nonfl-response',
       '64filler-response', 'Clareza', 'Atendimento', 'tempo_resposta'],
      dtype='object', length=134)
(31586, 134)
2    21682
0     9904
Name: Atendimento, dtype: int64
2    9338
0    4200
Name: Atendimento, dtype: int64


In [10]:
# Define pipeline and GridSearch CV
params = {    
    'clf_solver':'lbfgs',
    'clf_max_iter': 10000,
    #'clf_C': 100,
    'gs_cv': 10,
    'gs_scoring': 'f1_macro'   
}

pipelineClf = Pipeline([
    ('scaler', MaxAbsScaler()),
    ('clf', LogisticRegression(random_state=seed,
                               n_jobs=6,
                               class_weight='balanced',
                               # C=params.get('clf_C'),
                               solver=params.get('clf_solver'),
                               max_iter=params.get('clf_max_iter')))
])

gs_parameters = {
    'clf__C': (0.0001, 0.001, 0.01, 0.1, 1, 10, 100)
}

# Define grid search
grid_search_liwc = GridSearchCV(pipelineClf,
                               gs_parameters,
                               cv=params.get('gs_cv'),
                               scoring=params.get('gs_scoring'),
                               n_jobs=6,
                               verbose=10
                               )

In [11]:
X = X_train.iloc[:, 3: 3+64]
Y = y_train[label]
x = X_test.iloc[:, 3: 3+64]
y = y_test[label]

In [12]:
print("Executando Gridsearch para textos Solicitação Binary Unbalaneced - LIWC - Classe Atendimento")

now = str(datetime.now()).split('.')[0].replace('-', '_').replace(' ', '_').replace(':', '_')
print(now)

t0 = time()
grid_search_liwc.fit(X,Y)
print("done in %0.3fs" % (time() - t0))
print("Best score: %0.3f" % grid_search_liwc.best_score_)
print("Best parameters set:")
best_parameters = grid_search_liwc.best_estimator_.get_params()
for param_name in sorted(gs_parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Executando Gridsearch para textos Solicitação Binary Unbalaneced - LIWC - Classe Atendimento
2020_05_19_23_36_49
Fitting 10 folds for each of 7 candidates, totalling 70 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.2s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.3s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:    0.6s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:    0.9s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    1.4s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    2.3s
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:    5.1s
[Parallel(n_jobs=6)]: Done  67 out of  70 | elapsed:   13.8s remaining:    0.5s
[Parallel(n_jobs=6)]: Done  70 out of  70 | elapsed:   14.6s finished


done in 28.894s
Best score: 0.551
Best parameters set:
	clf__C: 100


In [13]:
# Saving Model
f_save = now + '_Atendimento_Solicitacao_Binary_Unbalanced_LIWC.sav'
pickle.dump(grid_search_liwc, open(os.path.join(save_path, f_save),'wb'))

# Saving Parameters
with open(os.path.join(save_path, 'params.txt'),'a') as f:
    f.write('\n\n' + ('#'*60))
    f.write('\n'+f_save + '\n\n')
    f.write('Parameters:\n')
    f.write(dumps(params) + '\n')
    f.write('\nGridSearch Best Parameters:\n')
    for param_name in sorted(gs_parameters.keys()):        
        f.write("%s: %r" % (param_name, best_parameters[param_name]) + '\n')    

# Validation

In [14]:
# Predictions
y_pred_train = grid_search_liwc.predict(X)
y_pred_test = grid_search_liwc.predict(x)

In [15]:
print('#'*50)
print('Report for TRAIN')
print('#'*50)
print(classification_report(Y, y_pred_train))

print('#'*50)
print('Report for TEST')
print('#'*50)
print(classification_report(y, y_pred_test))

##################################################
Report for TRAIN
##################################################
              precision    recall  f1-score   support

           0       0.38      0.63      0.48      9904
           2       0.76      0.54      0.63     21682

    accuracy                           0.57     31586
   macro avg       0.57      0.58      0.56     31586
weighted avg       0.64      0.57      0.58     31586

##################################################
Report for TEST
##################################################
              precision    recall  f1-score   support

           0       0.37      0.62      0.47      4200
           2       0.76      0.53      0.63      9338

    accuracy                           0.56     13538
   macro avg       0.57      0.58      0.55     13538
weighted avg       0.64      0.56      0.58     13538



In [24]:
with open(os.path.join(save_path, 'params.txt'),'a') as f:
    f.write('\n\n' + ('#'*60))
    f.write('\nReport for TRAIN')
    f.write('\n' + ('#'*60))
    f.write('\n' + classification_report(Y, y_pred_train))
    
    f.write('\n\n' + ('#'*60))
    f.write('\nReport for TEST')
    f.write('\n' + ('#'*60))
    f.write('\n' + classification_report(y, y_pred_test))